In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             confusion_matrix, classification_report,
                             precision_score, recall_score, f1_score)

In [2]:
# Load dataset
data = pd.read_csv('/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/data/cleaned_smushed_words.csv')

In [3]:
# Grab text exercise columns
text_columns = [f"text_exercise_{i}" for i in range(4, 20)] + ["text_exercise_final"]


# Define target and feature columns
target_columns = [
    "rating_chooses_appropriate_action", "rating_commits_to_action",
    "rating_gathers_information", "rating_identifies_issues_opportunities",
    "rating_interprets_information", "rating_involves_others",
    "rating_decision_making_final_score"
]

In [4]:
# Replace "NA" with "" in text columns and concatenate them
data = data.dropna(subset=target_columns)
data.loc[:, text_columns] = data.loc[:, text_columns].fillna('')
data['all_answers'] = data[text_columns].apply(lambda x: ' '.join(x), axis=1)

In [5]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['all_answers'], data[target_columns], test_size=0.2, random_state=42)

In [25]:
# Transform the text data using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [15]:
# Train ordinal classification models for each rating variable
ordinal_clf = MultiOutputClassifier(LogisticRegression(max_iter=1000), n_jobs=-1)
ordinal_clf.fit(X_train_transformed, y_train)

MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000), n_jobs=-1)

In [16]:
# Make predictions
y_pred = ordinal_clf.predict(X_test_transformed)

In [17]:
# Evaluate model performance - MAE and RMSE
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
mse = mean_squared_error(y_test, y_pred, multioutput='raw_values')
rmse = np.sqrt(mse)

In [18]:
print("Mean Absolute Error: ", mae)
print("Root Mean Squared Error: ", rmse)

Mean Absolute Error:  [0.31698113 0.34716981 0.52075472 0.37358491 0.38490566 0.46037736
 1.0490566 ]
Root Mean Squared Error:  [0.56301077 0.59558121 0.74225561 0.62344145 0.6672953  0.68405009
 1.50219965]


## What are MAE and RMSE, and what do they tell me?
The **Mean Absolute Error (MAE)** and **Root Mean Squared Error (RMSE)** are metrics used to evaluate the performance of a regression model. In this case, we have an ordinal classification model, which is a form of regression, so these metrics are appropriate.

**Mean Absolute Error (MAE)** is the average of the absolute differences between the predicted and actual values. It represents the average error magnitude without considering the direction. A *lower* MAE value indicates better performance of the model.

**Root Mean Squared Error (RMSE)** is the square root of the average of the squared differences between the predicted and actual values. RMSE gives more weight to larger errors, so it is more sensitive to outliers than MAE. A *lower* RMSE value indicates better performance of the model.

### Interpretation of MAE and RMSE results
The arrays correspond to the seven rating variables in the same order as they appear in the `target_columns` list. The first six rating variables are indicators of decision making, and the last variable is the final decision-making score.

The results says that the model has a lower error for the first six rating variables compared to the final decision-making score. Interesting. So, the model performs better at predicting the six indicators of decision making than predicting the overall final score 😑 The errors for the first six rating variables are in a similar range, which means the model's performance is pretty consistent across those variables.

Other notable information after researching (and help from chatGPT).  
- The first six rating variables have a scale of 1 to 4, while the final decision-making score has a scale of 1 to 7. So, the model has a decent performance for the first six rating variables. However, the final decision-making score has a much higher error.

In [21]:
# Evaluate model performance - Confusion Matrix, Classification Report, Precision, Recall, F1 Score
for i, target in enumerate(target_columns):
    print(f"Target column: {target}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test.iloc[:, i], y_pred[:, i]))
    print("Classification Report:")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))
    print(f"Precision: {precision_score(y_test.iloc[:, i], y_pred[:, i], average='weighted')}")
    print(f"Recall: {recall_score(y_test.iloc[:, i], y_pred[:, i], average='weighted')}")
    print(f"F1 Score: {f1_score(y_test.iloc[:, i], y_pred[:, i], average='weighted')}")
    print("\n\n")

Target column: rating_chooses_appropriate_action
Confusion Matrix:
[[ 25  51   0]
 [  8 156   0]
 [  0  25   0]]
Classification Report:
              precision    recall  f1-score   support

         2.0       0.76      0.33      0.46        76
         3.0       0.67      0.95      0.79       164
         4.0       0.00      0.00      0.00        25

    accuracy                           0.68       265
   macro avg       0.48      0.43      0.42       265
weighted avg       0.63      0.68      0.62       265

Precision: 0.6334023382819739
Recall: 0.6830188679245283
F1 Score: 0.619149081257442



Target column: rating_commits_to_action
Confusion Matrix:
[[  0   0   1   0]
 [  0   0  36   0]
 [  0   0 163   3]
 [  0   0  51  11]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        36
           3       0.65      0.98      0.78       166
           4       0

/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

## Summarize the Confusion Matrix, Classification Report, Precision, Recall, F1 Score results tell me?
**I love these illustrations!**

<div style="display: flex; flex-flow: row wrap; justify-content: space-around; ">
    <img src="https://miro.medium.com/v2/resize:fit:984/format:webp/1*f5ZeXvhsNFZ4q91M4Lotgg.jpeg" width=400px>
    <img src="https://miro.medium.com/v2/resize:fit:1018/format:webp/1*5CATnJ2FyNOF9xTpJ7qA5w.jpeg" width=400px>
</div>

The confusion matrix, classification report, and the calculated precision, recall, and F1 scores provide detailed information about the performance of the model for each target column (rating variable). Here is a summary of what the results tell you:  

- rating_chooses_appropriate_action: The model has an accuracy of 68.3%, with an F1 score of 61.9%. The confusion matrix shows that the model predicts class 3 quite well (95% recall), but struggles with class 4, having 0 true positives for it. The model's precision and recall are better for class 2 but still need improvement.

- rating_commits_to_action: The model has an accuracy of 65.7%, with an F1 score of 55.7%. The confusion matrix shows that the model does well with class 3 (98% recall) but struggles with the other classes. The model has low precision and recall for classes 1, 2, and 4.  

- rating_gathers_information: The model has an accuracy of 49.4%, with an F1 score of 44.9%. The confusion matrix indicates that the model does relatively well with class 3 (73% recall) but not so well with class 2, and it fails to predict class 1 and 4 correctly at all.  

- rating_identifies_issues_opportunities: The model has an accuracy of 63.4%, with an F1 score of 58.8%. The confusion matrix shows that the model predicts class 3 well (86% recall) but struggles with classes 1, 2, and 4, with low precision and recall.  

- rating_interprets_information: The model has an accuracy of 64.5%, with an F1 score of 59.7%. The confusion matrix reveals that the model predicts class 2 quite well (89% recall), but it has low precision and recall for class 3 and no true positive predictions for class 4.  

- rating_involves_others: The model has an accuracy of 54.3%, with an F1 score of 46.4%. The confusion matrix indicates that the model performs well with class 3 (87% recall), but struggles with classes 2 and 4, with low precision and recall for both.  

- rating_decision_making_final_score: The model has an accuracy of 38.1%, with an F1 score of 30.7%. The confusion matrix shows that the model predicts class 2 well (85% recall) but has low precision and recall for the other classes. The model does not make any true positive predictions for classes 1, 5, 6, and 7.  

Overall, the model seems to perform well for certain classes while struggling with others. To improve the model's performance, you could consider trying other classification algorithms, feature engineering, or addressing class imbalance using techniques like oversampling or undersampling.

In [30]:
# Comparing various models
# Define the models for comparison
models = [
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42, n_jobs=1)),
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=42, n_jobs=1)),
    ('SVM', SVC(random_state=42)),
    ('MultiOutput Logistic Regression', MultiOutputClassifier(LogisticRegression(max_iter=1000, random_state=42), n_jobs=1))
]

# Iterate through the models, train them, make predictions, and display the evaluation metrics
for name, model in models:
    # Check if the model is MultiOutput, in which case it does not need to be wrapped
    if name.startswith('MultiOutput'):
        clf = model
    else:
        clf = MultiOutputClassifier(model, n_jobs=1)
        
    # X_train_transformed.sort_indices()
    clf.fit(X_train_transformed, y_train)
    
    # Calculated score
    clf_score = clf.score(X_test_transformed, y_test)
    print("Classifier Accuracy: {:.2f}%".format(clf_score*100))

    # Make predictions
    y_pred = clf.predict(X_test_transformed)
    
    # Calculate metrics
    print(f"{name} Model Results:")
    for idx, target_col in enumerate(target_columns):
        print(f"Target column: {target_col}")
        print("Confusion Matrix:")
        print(confusion_matrix(y_test[target_col], y_pred[:, idx]))
        print("Classification Report:")
        print(classification_report(y_test[target_col], y_pred[:, idx]))
        
        precision = precision_score(y_test[target_col], y_pred[:, idx], average='weighted', zero_division=0)
        recall = recall_score(y_test[target_col], y_pred[:, idx], average='weighted', zero_division=0)
        f1 = f1_score(y_test[target_col], y_pred[:, idx], average='weighted', zero_division=0)
        
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 Score: {f1}\n")

Classifier Accuracy: 0.00%
Decision Tree Model Results:
Target column: rating_chooses_appropriate_action
Confusion Matrix:
[[  0   0   0   0]
 [  1  26  48   1]
 [  1  48 107   8]
 [  0   7  17   1]]
Classification Report:
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         0
         2.0       0.32      0.34      0.33        76
         3.0       0.62      0.65      0.64       164
         4.0       0.10      0.04      0.06        25

    accuracy                           0.51       265
   macro avg       0.26      0.26      0.26       265
weighted avg       0.49      0.51      0.49       265

Precision: 0.4864842171409379
Recall: 0.5056603773584906
F1 Score: 0.4945393468046994

Target column: rating_commits_to_action
Confusion Matrix:
[[ 0  0  1  0]
 [ 0  6 26  4]
 [ 0 29 98 39]
 [ 0  6 41 15]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
    

/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to cont

Classifier Accuracy: 2.26%
Random Forest Model Results:
Target column: rating_chooses_appropriate_action
Confusion Matrix:
[[ 17  59   0]
 [  8 156   0]
 [  0  25   0]]
Classification Report:
              precision    recall  f1-score   support

         2.0       0.68      0.22      0.34        76
         3.0       0.65      0.95      0.77       164
         4.0       0.00      0.00      0.00        25

    accuracy                           0.65       265
   macro avg       0.44      0.39      0.37       265
weighted avg       0.60      0.65      0.57       265

Precision: 0.5972830188679247
Recall: 0.6528301886792452
F1 Score: 0.5744815991033065

Target column: rating_commits_to_action
Confusion Matrix:
[[  0   0   1   0]
 [  0   2  33   1]
 [  0   1 155  10]
 [  0   0  46  16]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.67      0.06      0.10        36
           3       

/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

Classifier Accuracy: 0.75%
Logistic Regression Model Results:
Target column: rating_chooses_appropriate_action
Confusion Matrix:
[[ 25  51   0]
 [  8 156   0]
 [  0  25   0]]
Classification Report:
              precision    recall  f1-score   support

         2.0       0.76      0.33      0.46        76
         3.0       0.67      0.95      0.79       164
         4.0       0.00      0.00      0.00        25

    accuracy                           0.68       265
   macro avg       0.48      0.43      0.42       265
weighted avg       0.63      0.68      0.62       265

Precision: 0.6334023382819739
Recall: 0.6830188679245283
F1 Score: 0.619149081257442

Target column: rating_commits_to_action
Confusion Matrix:
[[  0   0   1   0]
 [  0   0  36   0]
 [  0   0 163   3]
 [  0   0  51  11]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        36
           3  

/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

Classifier Accuracy: 1.51%
SVM Model Results:
Target column: rating_chooses_appropriate_action
Confusion Matrix:
[[ 14  62   0]
 [  0 164   0]
 [  0  25   0]]
Classification Report:
              precision    recall  f1-score   support

         2.0       1.00      0.18      0.31        76
         3.0       0.65      1.00      0.79       164
         4.0       0.00      0.00      0.00        25

    accuracy                           0.67       265
   macro avg       0.55      0.39      0.37       265
weighted avg       0.69      0.67      0.58       265

Precision: 0.6911523716454935
Recall: 0.6716981132075471
F1 Score: 0.5783536662372762

Target column: rating_commits_to_action
Confusion Matrix:
[[  0   0   1   0]
 [  0   0  36   0]
 [  0   0 165   1]
 [  0   0  55   7]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        36
           3       0.64      

/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div

Classifier Accuracy: 0.75%
MultiOutput Logistic Regression Model Results:
Target column: rating_chooses_appropriate_action
Confusion Matrix:
[[ 25  51   0]
 [  8 156   0]
 [  0  25   0]]
Classification Report:
              precision    recall  f1-score   support

         2.0       0.76      0.33      0.46        76
         3.0       0.67      0.95      0.79       164
         4.0       0.00      0.00      0.00        25

    accuracy                           0.68       265
   macro avg       0.48      0.43      0.42       265
weighted avg       0.63      0.68      0.62       265

Precision: 0.6334023382819739
Recall: 0.6830188679245283
F1 Score: 0.619149081257442

Target column: rating_commits_to_action
Confusion Matrix:
[[  0   0   1   0]
 [  0   0  36   0]
 [  0   0 163   3]
 [  0   0  51  11]]
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00        36
  

/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/macuser/Documents/Coding/RStudio/SIOP_ML_2023_Discord/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_div